# Assigment: ATM Refilling Stochastic Problem
## Stochastic Programming (MESIO), 04/06/2025

### Arnau Pérez Reverte

In [1]:
from amplpy import AMPL, Environment
import pandas as pd

## Extensive form

In [2]:
ampl = AMPL(Environment(""))
ampl.read("ampl/atm.mod")
ampl.setOption("solver", "cplex")
ampl.option["solver"] = "cplex"

In [3]:
s = 7
c = 0.00025
q = 0.0011
l = 21000
u = 147000

ampl.param['s'] = s
ampl.param['c'] = c
ampl.param['q'] = q
ampl.param['l'] = l
ampl.param['u'] = u

SCENARIOS = list(range(1,8))

p = [0.04, 0.09, 0.10, 0.21, 0.27, 0.23, 0.06]
p = dict(zip(SCENARIOS, p))
xi = [150000, 120000, 110000, 100000, 80000, 60000, 50000]
xi = dict(zip(SCENARIOS, xi))

p_df = pd.DataFrame.from_dict(p, orient="index", columns=["p"])
p_df.index.name = 'SCENARIOS'
ampl.param['p'] = p_df


xi_df = pd.DataFrame.from_dict(xi, orient="index", columns=["p"])
xi_df.index.name = 'SCENARIOS'
ampl.param['xi'] = xi_df

In [5]:
print("=== EXTENSIVE FORM SOLUTION (Standard Form) ===")
ampl.solve("Extensive", solver="cplex")
extensive_obj = ampl.get_objective("Total_Cost").value()
extensive_x = {i: ampl.get_variable("x")[i].value() for i in range(1, 4)}
extensive_y = {i: ampl.get_variable("y")[i].value() for i in SCENARIOS}

print(f"Extensive form objective: {extensive_obj}")
print(f"Extensive form x variables: {extensive_x}")
print(f"Actual cash amount (x[1]): {extensive_x[1]}")
print(f"Verification: l ≤ x[1] ≤ u: {l} ≤ {extensive_x[1]} ≤ {u}")

=== EXTENSIVE FORM SOLUTION (Standard Form) ===
CPLEX 22.1.1CPLEX 22.1.1: optimal solution; objective 30.25
2 simplex iterations
Extensive form objective: 30.25
Extensive form x variables: {1: 110000.0, 2: 89000.0, 3: 37000.0}
Actual cash amount (x[1]): 110000.0
Verification: l ≤ x[1] ≤ u: 21000 ≤ 110000.0 ≤ 147000


## Benders' Decomposition Approach

In [10]:
ampl.reset()
ampl.read("ampl/atm.mod")
ampl.setOption("solver", "cplex")
#ampl.setOption('presolve', 0)
#ampl.set_option('gurobi_options', 'rays=1 presolve=0')
#ampl.set_option('gurobi_options', 'predual=0')

In [6]:
s = 7
c = 0.00025
q = 0.0011
l = 21000
u = 147000
s = 7

ampl.param['s'] = s
ampl.param['c'] = c
ampl.param['q'] = q
ampl.param['l'] = l
ampl.param['u'] = u
ampl.param['s'] = s

SCENARIOS = list(range(1,8))

p_values = [0.04, 0.09, 0.10, 0.21, 0.27, 0.23, 0.06]
xi_values = [150000, 120000, 110000, 100000, 80000, 60000, 50000]

p_df = pd.DataFrame({'p': p_values}, index=SCENARIOS)
p_df.index.name = 'SCENARIOS'
ampl.param['p'] = p_df

xi_df = pd.DataFrame({'xi': xi_values}, index=SCENARIOS)
xi_df.index.name = 'SCENARIOS'
ampl.param['xi'] = xi_df

In [8]:
n_cut = 0
eps = 1e-6
max_iter = 100

ampl.getParameter("n_cut").set(n_cut)

for iteration in range(max_iter):
    print(f"\n--- Iteration {iteration + 1} ---")
    
    # Solve master problem
    ampl.solve("Master", solver="cplex")
    master_obj = ampl.get_objective("Master_Obj").value()
    x_vals = {i: ampl.get_variable("x")[i].value() for i in range(1, 4)}
    
    print(f"Master objective: {master_obj}")
    print(f"x variables: {x_vals}")
    print(f"Actual cash (x[1]): {x_vals[1]}")
    
    # Fix x variables for subproblem
    ampl.param['x_fix'] = x_vals
    
    # Solve subproblem
    ampl.solve("Subproblem", solver="cplex")
    subproblem_status = ampl.get_value("solve_result")
    
    if subproblem_status == "solved":
        u_dual_values = ampl.get_variable("u_dual").get_values().to_dict()
        dual_obj = ampl.get_objective("Dual_Obj").value()
        
        upper_bound = c * x_vals[1] + dual_obj
        
        print(f"Dual objective: {dual_obj}")
        print(f"Upper bound: {upper_bound}")
        print(f"Gap: {abs(master_obj - upper_bound)}")
        
        # Check convergence
        if abs(master_obj - upper_bound) < eps:
            print(f"\n=== OPTIMAL SOLUTION FOUND ===")
            print(f"Optimal cash amount: {x_vals[1]}")
            print(f"Optimal objective: {master_obj}")
            print(f"Bound verification: {l} ≤ {x_vals[1]} ≤ {u}")
            break
        else:
            n_cut += 1
            ampl.param['n_cut'] = n_cut
            
            rhs = sum(u_dual_values[i] * xi_values[i-1] for i in SCENARIOS)
            coeff_x1 = c - sum(u_dual_values[i] for i in SCENARIOS)
            
            ampl.param['cuts_types'][n_cut] = "point"
            ampl.param['opt_cuts_rhs'][n_cut] = rhs
            ampl.param['opt_cuts_coeff'][n_cut, 1] = coeff_x1  
            ampl.param['opt_cuts_coeff'][n_cut, 2] = 0         
            ampl.param['opt_cuts_coeff'][n_cut, 3] = 0         
            
            print(f"Added optimality cut {n_cut}")
            
    elif "unbounded" in subproblem_status.lower():

        print("Subproblem unbounded - adding feasibility cut")
        n_cut += 1
        ampl.param['n_cut'] = n_cut
        
        # Get unbounded ray
        u_dual_ray = ampl.get_variable("u_dual").get_values("unbdd").to_dict()
        
        rhs = sum(u_dual_ray[i] * xi_values[i-1] for i in SCENARIOS)
        coeff_x1 = -sum(u_dual_ray[i] for i in SCENARIOS)
        
        ampl.param['cuts_types'][n_cut] = "ray"
        ampl.param['feas_cuts_rhs'][n_cut] = rhs
        ampl.param['feas_cuts_coeff'][n_cut, 1] = coeff_x1
        ampl.param['feas_cuts_coeff'][n_cut, 2] = 0
        ampl.param['feas_cuts_coeff'][n_cut, 3] = 0
        
        print(f"Added feasibility cut {n_cut}")
        
    else:
        print(f"Subproblem status: {subproblem_status}")
        break


--- Iteration 1 ---
CPLEX 22.1.1CPLEX 22.1.1: unbounded problem, no feasible solution returned
0 simplex iterations
 
------------ WARNINGS ------------
  Type                         MaxAbs [Name]   MaxRel [Name]
* algebraic con(s)             1E+05           1E+00         
*: Using the solver's aux variable values.
Documentation: mp.ampl.com/modeling-tools.html#automatic-solution-check.
Master objective: 0.0
x variables: {1: 110000.0, 2: 89000.0, 3: 37000.0}
Actual cash (x[1]): 110000.0
CPLEX 22.1.1CPLEX 22.1.1: optimal solution; objective 2.75
0 simplex iterations
Dual objective: 2.7500000000000004
Upper bound: 30.25
Gap: 30.25
Added optimality cut 1

--- Iteration 2 ---
CPLEX 22.1.1CPLEX 22.1.1: optimal solution; objective 20.727
1 simplex iterations
Master objective: 20.727
x variables: {1: 21000.0, 2: 0.0, 3: 126000.0}
Actual cash (x[1]): 21000.0
CPLEX 22.1.1CPLEX 22.1.1: optimal solution; objective 72.82
0 simplex iterations

"option abs_boundtol 1.3552527156068805e-20;"
or "op